# Import libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, BatchNormalization, Dropout, Input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime
from sklearn import preprocessing
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import plot_model
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
import pickle

Using TensorFlow backend.


# Load processed dataset
The data we load here are already shuffled and normalized.

In [2]:
data_directory = 'dataset/4. normalized-50/'
x_train = pickle.load(open(data_directory + "x_train.p", "rb"))
y_train = pickle.load(open(data_directory + "y_train.p", "rb"))
x_test = pickle.load(open(data_directory + "x_test.p", "rb"))
y_test = pickle.load(open(data_directory + "y_test.p", "rb"))
print('x_train: ' + str(x_train.shape))
print('y_train: ' + str(y_train.shape))
print('x_test: ' + str(x_test.shape))
print('y_test: ' + str(y_test.shape))

x_train: (604, 26)
y_train: (604, 2)
x_test: (604, 26)
y_test: (604, 2)


# Tunable parameters

In [3]:
epochs = 3000
batch_size = 20
learning_rate = 0.0001
decay = 0.0001

# Autoencoder #1

In [4]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
root_directory = 'results/30-stacked-autoencoder-50-test-size-01/'
model_directory = root_directory + "model/"
tensorboard_directory = root_directory + "tensorboard/"
def create_model(layers, dropout):
    global date
    x = Input(shape=(x_train.shape[1],))

    y = Dense(units = layers['layer_1'], activation='relu')(x)
    y = Dropout(dropout*0.75)(y)

    if (layers['layer_2'] > 0):
        y = Dense(units = layers['layer_2'], activation='relu')(y)
        y = Dropout(dropout)(y)
    
    y = Dense(units = x_train.shape[1], activation='sigmoid')(y)
    model = Model(x, y)
    
    # Create directory
    directory = model_directory + date + '/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Write model hyper-parameters
    df = pd.DataFrame({
        'Layer 1': layers['layer_1'],
        'Layer 2': layers['layer_2'],
        'Dropout 1': dropout*0.75,
        'Dropout 2': dropout,
        'Learning Rate': learning_rate,
        'decay': decay
    }, index=[0])
    df.to_csv(model_directory + date + '/params.csv')
    
    # Write model summary
    file2 = open(directory + "summary.txt", "a")
    model.summary(print_fn=lambda line: file2.write(line + '\n'))
    file2.close()

    # Write model diagram
    plot_model(model, to_file=directory + 'model.png', show_shapes=True, show_layer_names=False)
    
    # Compile the model
    model.compile(optimizer=keras.optimizers.Adam(lr=learning_rate, decay=decay), loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
    
    return model

# Prepare callbacks

In [5]:
class KerasClassifierTensorBoard(KerasClassifier):
    def fit(self, x, y, **kwargs):
        global date
        date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_directory + date + '/')
        csv_logger = keras.callbacks.CSVLogger(model_directory + date + '/epochs.csv')
        model_checkpoint = keras.callbacks.ModelCheckpoint(model_directory + date + '/weights.hdf5', save_best_only=True)
        callbacks = [tensorboard_callback, csv_logger, model_checkpoint]
        return super(KerasClassifierTensorBoard, self).fit(x, y, callbacks=callbacks, **kwargs)

# Grid search

In [6]:
model = KerasClassifierTensorBoard(build_fn=create_model, epochs=epochs, batch_size=batch_size)

layers = [
    {'layer_1': 10, 'layer_2': 8}
]
dropouts = [0.5, 0.3]
param_grid = dict(
    layers=layers, 
    dropout = dropouts
)

grid = GridSearchCV(estimator = model, param_grid = param_grid, error_score=0, verbose=2, cv=[(slice(None), slice(None))], n_jobs=1, fit_params=dict(validation_data=(x_test, x_test)))
grid_result = grid.fit(x_train, x_train)

Fitting 1 folds for each of 2 candidates, totalling 2 fits
[CV] dropout=0.5, layers={'layer_2': 8, 'layer_1': 10} ...............
Train on 604 samples, validate on 604 samples
Epoch 1/3000
604/604 [==============================] - 0s - loss: 18.0480 - acc: 0.1523 - val_loss: 17.8747 - val_acc: 0.0000e+00
Epoch 2/3000
604/604 [==============================] - 0s - loss: 18.0425 - acc: 0.1755 - val_loss: 17.8683 - val_acc: 0.0000e+00
Epoch 3/3000
604/604 [==============================] - 0s - loss: 18.0315 - acc: 0.1639 - val_loss: 17.8616 - val_acc: 0.0000e+00
Epoch 4/3000
604/604 [==============================] - 0s - loss: 18.0236 - acc: 0.1838 - val_loss: 17.8546 - val_acc: 0.0000e+00
Epoch 5/3000
604/604 [==============================] - 0s - loss: 18.0173 - acc: 0.2053 - val_loss: 17.8477 - val_acc: 0.0000e+00
Epoch 6/3000
604/604 [==============================] - 0s - loss: 18.0052 - acc: 0.1854 - val_loss: 17.8401 - val_acc: 0.0000e+00
Epoch 7/3000
604/604 [================

604/604 [==============================] - 0s - loss: 17.5344 - acc: 0.3974 - val_loss: 17.2418 - val_acc: 0.6838
Epoch 64/3000
604/604 [==============================] - 0s - loss: 17.5206 - acc: 0.4123 - val_loss: 17.2350 - val_acc: 0.6838
Epoch 65/3000
604/604 [==============================] - 0s - loss: 17.5076 - acc: 0.4007 - val_loss: 17.2282 - val_acc: 0.6838
Epoch 66/3000
604/604 [==============================] - 0s - loss: 17.5098 - acc: 0.4007 - val_loss: 17.2221 - val_acc: 0.6838
Epoch 67/3000
604/604 [==============================] - 0s - loss: 17.4880 - acc: 0.3957 - val_loss: 17.2167 - val_acc: 0.6838
Epoch 68/3000
604/604 [==============================] - 0s - loss: 17.4856 - acc: 0.4189 - val_loss: 17.2107 - val_acc: 0.6838
Epoch 69/3000
604/604 [==============================] - 0s - loss: 17.4960 - acc: 0.4272 - val_loss: 17.2048 - val_acc: 0.6838
Epoch 70/3000
604/604 [==============================] - 0s - loss: 17.4714 - acc: 0.4421 - val_loss: 17.1999 - val_ac

604/604 [==============================] - 0s - loss: 17.3014 - acc: 0.5795 - val_loss: 17.0110 - val_acc: 0.6838
Epoch 127/3000
604/604 [==============================] - 0s - loss: 17.3187 - acc: 0.5762 - val_loss: 17.0091 - val_acc: 0.6838
Epoch 128/3000
604/604 [==============================] - 0s - loss: 17.2877 - acc: 0.5828 - val_loss: 17.0072 - val_acc: 0.6838
Epoch 129/3000
604/604 [==============================] - 0s - loss: 17.3052 - acc: 0.5695 - val_loss: 17.0046 - val_acc: 0.6838
Epoch 130/3000
604/604 [==============================] - 0s - loss: 17.2839 - acc: 0.5844 - val_loss: 17.0021 - val_acc: 0.6838
Epoch 131/3000
604/604 [==============================] - 0s - loss: 17.2928 - acc: 0.5695 - val_loss: 16.9998 - val_acc: 0.6838
Epoch 132/3000
604/604 [==============================] - 0s - loss: 17.2893 - acc: 0.5811 - val_loss: 16.9971 - val_acc: 0.6838
Epoch 133/3000
604/604 [==============================] - 0s - loss: 17.2980 - acc: 0.5828 - val_loss: 16.9946 -

604/604 [==============================] - 0s - loss: 17.1877 - acc: 0.6308 - val_loss: 16.8986 - val_acc: 0.6838
Epoch 190/3000
604/604 [==============================] - 0s - loss: 17.1866 - acc: 0.6341 - val_loss: 16.8972 - val_acc: 0.6838
Epoch 191/3000
604/604 [==============================] - 0s - loss: 17.1797 - acc: 0.6391 - val_loss: 16.8960 - val_acc: 0.6838
Epoch 192/3000
604/604 [==============================] - 0s - loss: 17.2022 - acc: 0.6391 - val_loss: 16.8945 - val_acc: 0.6838
Epoch 193/3000
604/604 [==============================] - 0s - loss: 17.1907 - acc: 0.6291 - val_loss: 16.8933 - val_acc: 0.6838
Epoch 194/3000
604/604 [==============================] - 0s - loss: 17.1882 - acc: 0.6308 - val_loss: 16.8912 - val_acc: 0.6838
Epoch 195/3000
604/604 [==============================] - 0s - loss: 17.1832 - acc: 0.6275 - val_loss: 16.8892 - val_acc: 0.6838
Epoch 196/3000
604/604 [==============================] - 0s - loss: 17.1723 - acc: 0.6358 - val_loss: 16.8874 -

604/604 [==============================] - 0s - loss: 17.1271 - acc: 0.6325 - val_loss: 16.8162 - val_acc: 0.6838
Epoch 253/3000
604/604 [==============================] - 0s - loss: 17.1160 - acc: 0.6440 - val_loss: 16.8147 - val_acc: 0.6838
Epoch 254/3000
604/604 [==============================] - 0s - loss: 17.1037 - acc: 0.6407 - val_loss: 16.8137 - val_acc: 0.6838
Epoch 255/3000
604/604 [==============================] - 0s - loss: 17.1237 - acc: 0.6407 - val_loss: 16.8130 - val_acc: 0.6838
Epoch 256/3000
604/604 [==============================] - 0s - loss: 17.1157 - acc: 0.6407 - val_loss: 16.8117 - val_acc: 0.6838
Epoch 257/3000
604/604 [==============================] - 0s - loss: 17.1111 - acc: 0.6424 - val_loss: 16.8111 - val_acc: 0.6838
Epoch 258/3000
604/604 [==============================] - 0s - loss: 17.0934 - acc: 0.6457 - val_loss: 16.8098 - val_acc: 0.6838
Epoch 259/3000
604/604 [==============================] - 0s - loss: 17.1049 - acc: 0.6457 - val_loss: 16.8090 -

...

604/604 [==============================] - 0s - loss: 16.7510 - acc: 0.6507 - val_loss: 16.4486 - val_acc: 0.6921
Epoch 2708/3000
604/604 [==============================] - 0s - loss: 16.7547 - acc: 0.6440 - val_loss: 16.4487 - val_acc: 0.6921
Epoch 2709/3000
604/604 [==============================] - 0s - loss: 16.7423 - acc: 0.6540 - val_loss: 16.4487 - val_acc: 0.6921
Epoch 2710/3000
604/604 [==============================] - 0s - loss: 16.7727 - acc: 0.6507 - val_loss: 16.4487 - val_acc: 0.6921
Epoch 2711/3000
604/604 [==============================] - 0s - loss: 16.7364 - acc: 0.6341 - val_loss: 16.4486 - val_acc: 0.6921
Epoch 2712/3000
604/604 [==============================] - 0s - loss: 16.7356 - acc: 0.6540 - val_loss: 16.4487 - val_acc: 0.6921
Epoch 2713/3000
604/604 [==============================] - 0s - loss: 16.7481 - acc: 0.6440 - val_loss: 16.4487 - val_acc: 0.6921
Epoch 2714/3000
604/604 [==============================] - 0s - loss: 16.7414 - acc: 0.6374 - val_loss: 16

604/604 [==============================] - 0s - loss: 16.7506 - acc: 0.6474 - val_loss: 16.4470 - val_acc: 0.6921
Epoch 2771/3000
604/604 [==============================] - 0s - loss: 16.7488 - acc: 0.6391 - val_loss: 16.4470 - val_acc: 0.6921
Epoch 2772/3000
604/604 [==============================] - 0s - loss: 16.7266 - acc: 0.6474 - val_loss: 16.4471 - val_acc: 0.6921
Epoch 2773/3000
604/604 [==============================] - 0s - loss: 16.7390 - acc: 0.6474 - val_loss: 16.4470 - val_acc: 0.6921
Epoch 2774/3000
604/604 [==============================] - 0s - loss: 16.7507 - acc: 0.6474 - val_loss: 16.4469 - val_acc: 0.6921
Epoch 2775/3000
604/604 [==============================] - 0s - loss: 16.7495 - acc: 0.6523 - val_loss: 16.4469 - val_acc: 0.6921
Epoch 2776/3000
604/604 [==============================] - 0s - loss: 16.7150 - acc: 0.6490 - val_loss: 16.4469 - val_acc: 0.6921
Epoch 2777/3000
604/604 [==============================] - 0s - loss: 16.7568 - acc: 0.6424 - val_loss: 16

604/604 [==============================] - 0s - loss: 16.7439 - acc: 0.6523 - val_loss: 16.4467 - val_acc: 0.6921
Epoch 2834/3000
604/604 [==============================] - 0s - loss: 16.7302 - acc: 0.6440 - val_loss: 16.4466 - val_acc: 0.6921
Epoch 2835/3000
604/604 [==============================] - 0s - loss: 16.7465 - acc: 0.6474 - val_loss: 16.4466 - val_acc: 0.6921
Epoch 2836/3000
604/604 [==============================] - 0s - loss: 16.7740 - acc: 0.6490 - val_loss: 16.4466 - val_acc: 0.6921
Epoch 2837/3000
604/604 [==============================] - 0s - loss: 16.7267 - acc: 0.6507 - val_loss: 16.4465 - val_acc: 0.6921
Epoch 2838/3000
604/604 [==============================] - 0s - loss: 16.7566 - acc: 0.6523 - val_loss: 16.4464 - val_acc: 0.6921
Epoch 2839/3000
604/604 [==============================] - 0s - loss: 16.7198 - acc: 0.6474 - val_loss: 16.4464 - val_acc: 0.6921
Epoch 2840/3000
604/604 [==============================] - 0s - loss: 16.7510 - acc: 0.6424 - val_loss: 16

604/604 [==============================] - 0s - loss: 16.7306 - acc: 0.6407 - val_loss: 16.4457 - val_acc: 0.6921
Epoch 2897/3000
604/604 [==============================] - 0s - loss: 16.7078 - acc: 0.6440 - val_loss: 16.4457 - val_acc: 0.6921
Epoch 2898/3000
604/604 [==============================] - 0s - loss: 16.7317 - acc: 0.6474 - val_loss: 16.4457 - val_acc: 0.6921
Epoch 2899/3000
604/604 [==============================] - 0s - loss: 16.7427 - acc: 0.6474 - val_loss: 16.4456 - val_acc: 0.6921
Epoch 2900/3000
604/604 [==============================] - 0s - loss: 16.7472 - acc: 0.6523 - val_loss: 16.4458 - val_acc: 0.6921
Epoch 2901/3000
604/604 [==============================] - 0s - loss: 16.7412 - acc: 0.6540 - val_loss: 16.4460 - val_acc: 0.6921
Epoch 2902/3000
604/604 [==============================] - 0s - loss: 16.7364 - acc: 0.6440 - val_loss: 16.4459 - val_acc: 0.6921
Epoch 2903/3000
604/604 [==============================] - 0s - loss: 16.7315 - acc: 0.6507 - val_loss: 16

604/604 [==============================] - 0s - loss: 16.7430 - acc: 0.6523 - val_loss: 16.4453 - val_acc: 0.6921
Epoch 2960/3000
604/604 [==============================] - 0s - loss: 16.7095 - acc: 0.6507 - val_loss: 16.4453 - val_acc: 0.6921
Epoch 2961/3000
604/604 [==============================] - 0s - loss: 16.7198 - acc: 0.6556 - val_loss: 16.4452 - val_acc: 0.6921
Epoch 2962/3000
604/604 [==============================] - 0s - loss: 16.7621 - acc: 0.6523 - val_loss: 16.4451 - val_acc: 0.6921
Epoch 2963/3000
604/604 [==============================] - 0s - loss: 16.7514 - acc: 0.6341 - val_loss: 16.4452 - val_acc: 0.6921
Epoch 2964/3000
604/604 [==============================] - 0s - loss: 16.7515 - acc: 0.6457 - val_loss: 16.4451 - val_acc: 0.6921
Epoch 2965/3000
604/604 [==============================] - 0s - loss: 16.7246 - acc: 0.6523 - val_loss: 16.4451 - val_acc: 0.6921
Epoch 2966/3000
604/604 [==============================] - 0s - loss: 16.7418 - acc: 0.6407 - val_loss: 16